In [1]:
import gymnasium as gym
from scripts.DQN_Optimizer import DQN_Optimizer

In [2]:
env = gym.make("CartPole-v1")

In [3]:
param_dict = {
                "batch_size" : 128,
                "gamma" : 0.99,
                "eps_start" : 1,
                "eps_end" : 0.1,
                "decay_steps" : 100000,
                "tau" : 0.005,
                "learning_rate" : 1e-4,
            }

dqn_opt = DQN_Optimizer(env = env, replay_memory_size= 100, param_dict=param_dict,num_episodes=50)

In [4]:
dqn_opt.set_optimizer("adam")

In [5]:
dqn_opt.run_optimization()

Current epsilon: 0.99: 100%|██████████| 50/50 [00:07<00:00,  6.89it/s]

Optimization complete


In [6]:
dqn_opt.episode_cumulative_reward

[10.0,
 12.0,
 12.0,
 22.0,
 20.0,
 24.0,
 23.0,
 16.0,
 23.0,
 15.0,
 21.0,
 16.0,
 16.0,
 98.0,
 26.0,
 20.0,
 42.0,
 15.0,
 23.0,
 9.0,
 13.0,
 13.0,
 10.0,
 21.0,
 57.0,
 27.0,
 11.0,
 29.0,
 18.0,
 21.0,
 13.0,
 34.0,
 12.0,
 18.0,
 9.0,
 18.0,
 34.0,
 11.0,
 14.0,
 13.0,
 13.0,
 14.0,
 20.0,
 17.0,
 21.0,
 15.0,
 13.0,
 16.0,
 21.0,
 15.0]

In [ ]:
dqn_opt.plot_durations()